In [1]:
import pandas as pd, glob, numpy as np
from sklearn import preprocessing as scale

Collecting repeat data (percentages of genome), putting in dictionary

In [2]:
fish_dictionary = {}
for f in glob.glob('../data/RepeatMaskerTables/*tbl'):
    fish = f.split('/')[-1].split('.')[0]
    
    if fish == 'Final_PilonAsm_ssalBroken':
        fish = 'fish_145'
    fh = open(f, 'r')
    tbl = fh.readlines()
    fh.close()
    class_dictionary = {}
    
    for Class in ['LINEs', 'SINEs', 'DNA elements', 'LTR elements', 'Simple repeats', 'Total interspersed repeats']:
        c = [i for i in tbl if Class in i][0].split()[-0].split(':')[0]
        p = [i for i in tbl if Class in i][0].split()[-2]
        class_dictionary.update({ c : float(p)})
    fish_dictionary.update({fish : class_dictionary})

names of fish

In [3]:
fishinfo = pd.read_csv('../data/metadata/names.tab', sep = '\t')
fishinfo.COMMON_NAME = [x.replace('0xb4', '') for x in fishinfo.COMMON_NAME]
fishinfo.COMMON_NAME = [x.replace('\xc2\xb4s', '') for x in fishinfo.COMMON_NAME]
fishinfo.COMMON_NAME = [x.replace('\'', '').replace(' ', '_') for x in fishinfo.COMMON_NAME]
alias_common_name = fishinfo.set_index('ALIAS')['COMMON_NAME'].to_dict()
alias_family = fishinfo.set_index('ALIAS')['FAMILY'].to_dict()
common_name_family = fishinfo.set_index('COMMON_NAME')['FAMILY'].to_dict()
alias_species = dict(fishinfo.set_index('ALIAS')['SPECIES'])
fishinfo.head()

,ALIAS,FAMILY,ORDER,SPECIES,COMMON_NAME
0,fish_1,Gadidae,Gadiformes,Arctogadus glacialis,Arctic_cod
1,fish_2,Gadidae,Gadiformes,Boreogadus saida,Polar_cod
2,fish_3,Gadidae,Gadiformes,Trisopterus minutus,Poor_cod
3,fish_4,Gadidae,Gadiformes,Pollachius virens,Saithe
4,fish_5,Gadidae,Gadiformes,Melanogrammus aeglefinus,Haddoc


phobos simple repeat data along with assembly stats

In [4]:
all_species_stats = pd.read_csv('../data/PhobosData/all_species_7th_august.stats.tsv', 
                                sep = '\t').rename(columns = {'species' : 'ALIAS'})
all_species_stats.head()

,ALIAS,bp,contig_bp,n50_contig,n50_scaffold,longest_scaffold,num_strs_unit20,mean_length_unit20,mean_length_sd_unit20,strs_bp_per_mbp_unit20,...,strs_bp_per_mbp_unit10_9,strs_corr_bp_per_mbp_unit10_9,strs_per_mbp_unit10_9,num_strs_unit10_10,mean_length_unit10_10,mean_length_sd_unit10_10,strs_bp_per_mbp_unit10_10,strs_corr_bp_per_mbp_unit10_10,strs_per_mbp_unit10_10,preqc_genome_size
0,fish_120,1018701900,884507431,1757,59657,2047057,620971,27.77,31.78,16629.50,...,275.37,317.14,8.60511,16597,42.19,46.86,672.26,774.26,16.30350,0
1,fish_121,1151137423,1024041585,2179,52849,1141856,795274,29.77,36.68,20107.60,...,348.35,391.58,10.10860,25152,45.19,53.47,962.66,1082.14,21.85230,0
2,fish_122,831411547,698949643,21886,1194190,7036168,259023,27.93,25.21,8602.74,...,188.28,223.96,6.93381,7209,32.77,42.20,281.91,335.34,8.67509,0
3,fish_123,1191242572,964234932,13755,1775308,9823298,752217,32.07,40.46,19822.40,...,532.65,658.05,17.31650,32952,42.22,57.96,1142.98,1412.07,27.66750,0
4,fish_124,955735039,889887957,20438,2375582,17765049,661679,70.93,184.90,45968.60,...,1677.86,1802.02,24.44820,94839,131.10,231.57,12148.70,13047.70,99.30790,0


raw numbers of repetitive sequences

In [ ]:
dat = '../data/RepeatMaskerOut/*out'

def returnNumberOfElements(TE_class, f):
    out = !grep $TE_class $f | grep -v "*" | wc -l
    return(int(out[0]))

fishes = []
dnas = []
lines = []
sines = []
ltrs = []

for f in glob.glob(dat):
    fish = f.split('/')[-1].split('.')[0]
    if fish == 'Final_PilonAsm_ssalBroken':
        fish = 'fish_145'
    fishes.append(fish)
    dna = returnNumberOfElements('DNA', f)
    ltr = returnNumberOfElements('LTR', f)
    sine = returnNumberOfElements('SINE', f)
    line = returnNumberOfElements('LINE', f)
    dnas.append(dna)
    ltrs.append(ltr)
    sines.append(sine)
    lines.append(line)

In [ ]:
n_TEs = pd.DataFrame([fishes, 
              dnas, 
              lines, 
              sines, 
              ltrs]).T.rename(columns = 
                              {0 : 'ALIAS', 
                               1 : 'n_DNA', 
                               2 : 'n_LTR', 
                               3 : 'n_SINE', 
                               4 : 'n_LINE'})#.set_index('ALIAS')

In [ ]:
n_TEs.drop_duplicates(subset = 'ALIAS').set_index('ALIAS').to_csv('../data/RepeatMaskerRawNumbers/n_TEs')

In [9]:
nTEs = pd.read_csv('../data/RepeatMaskerRawNumbers/n_TEs')
nTEs.head()

,ALIAS,n_DNA,n_LTR,n_SINE,n_LINE
0,fish_145,625625,262719,52993,69978
1,fish_1,174989,40077,11334,45839
2,fish_10,180891,26833,6926,44553
3,fish_100,158025,32186,3761,50240
4,fish_101,759558,73184,29482,102368


In [10]:
fishbase = pd.read_csv('../data/FishBase/FishBaseStats.tsv', sep = '\t')

In [11]:
fishbase.head()

,species,freshwater,brackwater,saltwater,brood1,brood2
0,Anguilla anguilla,1,1,1,nonguarders,open water/substratum egg scatterers
1,Anguilla japonica,1,1,0,nonguarders,open water/substratum egg scatterers
2,Astatotilapia burtoni,1,0,0,bearers,external brooders
3,Astyanax mexicanus,1,0,0,nonguarders,brood hiders
4,Boleophthalmus pectinirostris,1,1,1,na,na


making final DataFrame

In [ ]:
# Making DataFrame

# percentages of TEs
Repeats = pd.DataFrame(fish_dictionary).T.reset_index()
Repeats['TEs'] = [x.DNA + x.LINEs + x.LTR + x.SINEs for x in Repeats.itertuples()]
Repeats['Common_name'] = [alias_common_name[x] for x in Repeats['index']]
Repeats['latin'] = [alias_species[x].replace(' ', '_') for x in Repeats['index']]
Repeats['FAMILY'] = [alias_family[x] for x in Repeats['index']]
Repeats = Repeats.rename(columns = {'index' : 'ALIAS'})
# Fraction of total
Repeats['DNA_frac'] = [x.DNA / x.TEs for x in Repeats.itertuples()]
Repeats['LINEs_frac'] = [x.LINEs / x.TEs for x in Repeats.itertuples()]
Repeats['SINEs_frac'] = [x.SINEs / x.TEs for x in Repeats.itertuples()]
Repeats['LTR_frac'] = [x.LTR / x.TEs for x in Repeats.itertuples()]

# phobos and assembly stats
Repeats = pd.merge(Repeats, all_species_stats, on = 'ALIAS')
Repeats['strs_perc'] = (Repeats['strs_bp_per_mbp_unit10'] / 1e6) * 100

# Proportions of strs
Repeats['sum_strs_per_mbp_unit10'] = Repeats.strs_bp_per_mbp_unit10_1 + Repeats.strs_bp_per_mbp_unit10_2 \
+ Repeats.strs_bp_per_mbp_unit10_3 + Repeats.strs_bp_per_mbp_unit10_4 + Repeats.strs_bp_per_mbp_unit10_5 + \
Repeats.strs_bp_per_mbp_unit10_6 + Repeats.strs_bp_per_mbp_unit10_7 + Repeats.strs_bp_per_mbp_unit10_8 + \
Repeats.strs_bp_per_mbp_unit10_9 + Repeats.strs_bp_per_mbp_unit10_10

# fishbase stats
fishbase['latin'] = [x.replace(' ', '_') for x in fishbase.species]

Repeats = pd.merge(fishbase, Repeats, on = 'latin')
Repeats['prop_class'] = Repeats.TEs / Repeats.Total
Repeats['strs_perc'] = (Repeats['strs_bp_per_mbp_unit10'] / 1e6) * 100
Repeats['log_strs'] = np.log10(Repeats['strs_bp_per_mbp_unit10'])
Repeats['brood1_cat'] = ['b_g' if x in ['bearers', 'guarders'] else x for x in Repeats.brood1]
Repeats['brood1_cat'] = ['ng' if x in ['nonguarders'] else x for x in Repeats.brood1_cat]

# fresh/brack/salt water to numbers
fresh_salt_c = []
fresh_salt_bin = []
for row in Repeats.itertuples():
    if row.freshwater == 0 and row.saltwater == 1:
        fresh_salt_c.append('salt')
        fresh_salt_bin.append(0)
    elif row.freshwater == 1 and row.saltwater == 0:
        fresh_salt_c.append('fresh')
        fresh_salt_bin.append(1)
    else:
        fresh_salt_c.append(np.nan)
        fresh_salt_bin.append(np.nan)
Repeats['water'] = fresh_salt_c
Repeats['water_bin'] = fresh_salt_bin

# combining water and guarding
fw_g = list(Repeats[(Repeats.water == 'fresh') & (Repeats.brood1_cat == 'b_g')].ALIAS)
sw_ng = list(Repeats[(Repeats.water == 'salt') & (Repeats.brood1_cat == 'ng')].ALIAS)

w_b = []
for x in Repeats.ALIAS:
    if x in fw_g:
        w_b.append(1)
    elif x in sw_ng:
        w_b.append(0)
    else:
        w_b.append(np.nan)
Repeats['w_b'] = w_b

# raw numbers
Repeats = pd.merge(nTEs, Repeats, on = 'ALIAS')
Repeats['nTEs'] = Repeats.n_DNA + Repeats.n_LINE + Repeats.n_LTR + Repeats.n_SINE
Repeats['nTEs_density'] = (Repeats.nTEs / Repeats.bp) * 1000000

Repeats = Repeats.set_index('ALIAS').replace('na', np.nan)
Repeats.to_csv('../data/DataFrame/Repeats.tsv', sep = '\t', na_rep = np.nan)

family specific net div rates

In [79]:
treeoflife = pd.read_excel('../data/DiversificationRates/rspb20161334supp2.xls').rename(columns = {'family' : 'FAMILY'})
family_div_rates = treeoflife[treeoflife['FAMILY'].isin(set(fishinfo.FAMILY))][['FAMILY', 'div_rate_e5_family']]
treeoflife.head()

,kingdom,FAMILY,family_age,family_age_ln,family_richness,family_richness_ln,taxon name in tree,div_rate_e0_family,div_rate_e5_family,div_rate_e9_family
0,Animal,Bourletiellidae,103.74600,4.641946,245,5.501258,CollBourl,0.053026,0.046384,0.031180
1,Animal,Brachystomellidae,56.52618,4.034704,129,4.859812,CollBrach,0.085975,0.073849,0.046433
2,Animal,Dicyrtomidae,160.57450,5.078758,200,5.298317,CollDicyr,0.032996,0.028710,0.018930
3,Animal,Entomobryidae,104.88340,4.652849,1678,7.425358,CollEntom,0.070796,0.064193,0.048894
4,Animal,Hypogastruridae,105.99640,4.663405,682,6.525030,CollHypog,0.061559,0.055033,0.039959


In [80]:
RepeatsGrouped = pd.merge(Repeats, treeoflife, on = 'FAMILY').groupby('FAMILY').median()
RepeatsGrouped.to_csv('../data/DataFrame/RepeatsGrouped.tsv', sep = '\t')